<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/No_supervisados_Ejercicio_3__kmeans_penguins_resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# No supervisados - Ejercicio 3: kmeans_penguins.ipynb

Este notebook es **You do**. Trabajarás de forma autónoma aplicando KMeans (con comparación a DBSCAN), PCA y LOF sobre el dataset Palmer Penguins. Sigue las instrucciones en cada celda, ejecuta el código y escribe tus conclusiones al final.

## Objetivos

- Cargar y explorar el dataset `penguins`.
- Preparar los datos (limpieza, encoding, escalado), aplicar KMeans y comparar con DBSCAN.
- Usar PCA para visualización y LOF para detección de outliers; interpretar resultados y redactar conclusiones.

## Descripción del dataset

Palmer Penguins contiene medidas morfológicas de pingüinos (`bill_length_mm`, `bill_depth_mm`, `flipper_length_mm`, `body_mass_g`) y variables categóricas (`species`, `island`, `sex`). Cárgalo con `seaborn.load_dataset('penguins')` o desde el raw CSV si no está disponible.

In [ ]:
# 1) Importaciones
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
from sklearn.metrics import silhouette_score, adjusted_rand_score
np.random.seed(42)

### Carga de datos

Carga el dataset `penguins`. Si `seaborn` no proporciona el dataset en tu entorno, usa la versión raw en GitHub.

In [ ]:
try:
    df = sns.load_dataset('penguins')
except Exception:
    url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv'
    df = pd.read_csv(url)
display(df.head())
print('Shape:', df.shape)

### Exploración inicial

Revisa estructura, nulos y distribuciones. Ejecuta las celdas y observa los resultados.

In [ ]:
print(df.info())
display((df.isnull().mean()*100).round(2))

# Distribución de categorías
print('\nDistribución por species:')
print(df['species'].value_counts())
print('\nDistribución por sex:')
print(df['sex'].value_counts())

# Pairplot sugerido para variables numéricas
sns.pairplot(df[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g','species']].dropna(), hue='species')
plt.show()

### Selección de variables y limpieza

Decide si imputas o eliminas filas con nulos. Selecciona las variables numéricas y, si vas a usar categóricas, codifícalas con OneHotEncoder.

In [ ]:
# Selección según la convención del curso
vars_numericas = df.select_dtypes(include=[np.number]).columns
vars_categoricas = df.select_dtypes(include=['object','category']).columns
print('Numéricas:', list(vars_numericas))
print('Categóricas:', list(vars_categoricas))

# Ejemplo simple: eliminar filas con nulos en numéricas
df_clean = df.dropna(subset=vars_numericas).reset_index(drop=True)
display(df_clean.head())
print('New shape:', df_clean.shape)

### Preprocesado: encoding y escalado

Codifica las categóricas (si las usas) y escala todas las variables numéricas con StandardScaler. La idea es obtener una matriz `X_scaled` lista para modelado.

In [ ]:
# Pipeline de ejemplo (ajusta si decides incluir categóricas)
X_num = df_clean[vars_numericas]
if len(vars_categoricas) > 0:
    ohe = OneHotEncoder(sparse=False, drop='first')
    X_cat = ohe.fit_transform(df_clean[vars_categoricas])
    X_full = np.concatenate([X_num.values, X_cat], axis=1)
else:
    X_full = X_num.values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_full)
print('X_scaled shape:', X_scaled.shape)

### PCA para visualización y compresión

Calcula PCA para visualizar en 2D y para estimar cuántas componentes retienen la mayor parte de la varianza.

In [ ]:
pca = PCA(n_components=2)
X_pca2 = pca.fit_transform(X_scaled)
print('Explained variance ratio (2 PCs):', pca.explained_variance_ratio_)
plt.figure(); plt.bar([1,2], pca.explained_variance_ratio_); plt.title('Explained variance (2 PCs)'); plt.show()

# PCA para 90% varianza (opcional)
pca90 = PCA(n_components=0.90)
X_pca90 = pca90.fit_transform(X_scaled)
print('n_components to retain 90% variance:', getattr(pca90, 'n_components_', None))

### KMeans

Prueba manualmente varios valores de `k` (por ejemplo 2..8). Calcula silhouette score para cada `k` y elige un valor razonable.

In [ ]:
results = []
for k in range(2,9):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    results.append({'k':k, 'silhouette': score, 'inertia': km.inertia_})
res_df = pd.DataFrame(results).set_index('k')
display(res_df)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(res_df.index, res_df['inertia'], marker='o'); plt.title('Inertia');
plt.subplot(1,2,2); plt.plot(res_df.index, res_df['silhouette'], marker='o'); plt.title('Silhouette');
plt.show()

k_chosen = int(res_df['silhouette'].idxmax())
print('k elegido (máx silhouette):', k_chosen)

km = KMeans(n_clusters=k_chosen, random_state=42, n_init=10)
labels_km = km.fit_predict(X_scaled)
df_clean['cluster_km'] = labels_km
plt.figure(figsize=(8,6)); sns.scatterplot(x=X_pca2[:,0], y=X_pca2[:,1], hue=labels_km.astype(str), palette='tab10'); plt.title('KMeans clusters (PCA 2D)'); plt.show()

### Comparación con DBSCAN

Usa un k-distance plot para estimar `eps` y prueba DBSCAN con valores diferentes de `eps` y `min_samples`. Compara las etiquetas con KMeans usando adjusted rand index si procede.

In [ ]:
neigh = NearestNeighbors(n_neighbors=5)
distances, indices = neigh.fit(X_scaled).kneighbors(X_scaled)
distances = np.sort(distances[:, -1])
plt.plot(distances); plt.title('k-distance plot'); plt.show()

# Ejecuta DBSCAN con un ejemplo de parámetros
from sklearn.metrics import adjusted_rand_score

db = DBSCAN(eps=0.5, min_samples=5)
labels_db = db.fit_predict(X_scaled)
df_clean['cluster_db'] = labels_db
print('Unique labels (DBSCAN):', np.unique(labels_db))
print(pd.Series(labels_db).value_counts())
print('Adjusted Rand Index (KMeans vs DBSCAN):', adjusted_rand_score(labels_km, labels_db))
plt.figure(figsize=(8,6)); sns.scatterplot(x=X_pca2[:,0], y=X_pca2[:,1], hue=labels_db.astype(str), palette='tab10'); plt.title('DBSCAN labels (PCA 2D)'); plt.show()

### LOF — detección de outliers

Aplica LocalOutlierFactor para identificar observaciones atípicas. Ajusta `n_neighbors` y `contamination` según convenga.

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_labels = lof.fit_predict(X_scaled)
df_clean['lof_label'] = lof_labels
print('Counts LOF labels:', pd.Series(lof_labels).value_counts())
plt.figure(figsize=(8,6)); sns.scatterplot(x=X_pca2[:,0], y=X_pca2[:,1], hue=lof_labels.astype(str), palette='tab10'); plt.title('LOF labels (PCA 2D)'); plt.show()

# Comparar outliers LOF con ruido DBSCAN
print('\nEjemplos where DBSCAN noise (-1):')
display(df_clean[df_clean['cluster_db']==-1].head())
print('\nEjemplos detected by LOF (-1):')
display(df_clean[df_clean['lof_label']==-1].head())

### Perfilado y acciones

Analiza medias por cluster y las características de los outliers. Decide si excluir, investigar o tratar estos casos de forma específica.

In [ ]:
print('Perfil KMeans:')
display(df_clean.groupby('cluster_km')[vars_numericas].mean().round(2))

print('\nOutliers LOF examples:')
display(df_clean[df_clean['lof_label']==-1].head())

### Conclusiones

Escribe aquí un párrafo con tus conclusiones: elección de k o parámetros de DBSCAN, qué encontró LOF y recomendaciones prácticas.

In [ ]:
# Escribe tu conclusión como texto en esta celda, por ejemplo:
conclusion = '''
'''
print(conclusion)

### Guardar notebook

Ejecuta todas las celdas y guarda el notebook. Luego descarga el archivo para entregar o subir a tu repositorio.